# 数据集的划分

In [1]:
import pandas as pd
import os
import shutil
from tqdm import tqdm

In [8]:
# 标注文件路径 (请确保这是包含 dataset_split 列的文件)
csv_path = r"X:\\数据集\\DeepShip\\data_preprocessing\\annotation\\DeepShip_No_Overlap_Metadata_add_Environmental_Noise.csv"
# 输入音频文件目录
SOURCE_ROOT = r"X:\\数据集\DeepShip\\data_preprocessing\\data_audio_rename_add_enviromental_noise_class"

# 输出目录 (程序会自动创建 train 和 test 文件夹)
OUTPUT_ROOT = r"X:\\数据集\\DeepShip\\data_preprocessing\\data_audio_rename_add_enviromental_noise_class_dataset_split"

In [7]:
def build_file_index(root_path):
    """
    不管文件是在 Cargo/0_1.wav 还是 Background/test/3_0/0000/4_0_1.wav
    只要在这个根目录下，都会被记录下来。
    返回字典: {'文件名': '完整绝对路径'}
    """
    print(f"🔍 正在扫描源目录建立索引: {root_path}")
    print("   (如果文件很多，这可能需要几秒钟...)")
    
    file_map = {}
    count = 0
    for root, dirs, files in os.walk(root_path):
        for file in files:
            if file.endswith('.wav'):
                # 记录: 文件名 -> 完整路径
                file_map[file] = os.path.join(root, file)
                count += 1
                
    print(f"✅ 索引建立完成！共找到 {count} 个音频文件。")
    return file_map

In [ ]:
def split_dataset_unified():
    # 读取 CSV
    if not os.path.exists(csv_path):
        print(f"❌ 错误: 找不到 CSV 文件 {csv_path}")
        return
    df = pd.read_csv(csv_path)
    
    # 只处理 train 和 test
    df_valid = df[df['dataset_split'].isin(['train', 'test'])]
    print(f"📋 待处理任务总数: {len(df_valid)}")

    # 建立文件索引 (解决路径深浅不一的问题)
    file_index = build_file_index(SOURCE_ROOT)
    
    if len(file_index) == 0:
        print("❌ 严重错误: 源目录下没有扫描到任何 .wav 文件！请检查路径。")
        return

    # 3.3 类别映射
    id_to_folder = {
        0: 'Cargo', 
        1: 'Passengership', 
        2: 'Tanker', 
        3: 'Tug', 
        4: 'Background'
    }

    success_count = 0
    missing_count = 0
    
    # 使用 tqdm 显示进度条
    for index, row in tqdm(df_valid.iterrows(), total=len(df_valid), desc="Splitting"):
        class_id = row['class_id']
        filename = row['new_filename'] # 例如 0_1.wav 或 4_0_1.wav
        split = row['dataset_split']   # train 或 test
        
        # 获取目标子文件夹名
        folder_name = id_to_folder.get(class_id)
        if not folder_name: continue

        # --- A. 查找源文件 (Source) ---
        # 直接查字典，不用管它在哪个子文件夹里
        src_path = file_index.get(filename)
        
        if not src_path:
            # 字典里没查到，说明文件真的不存在
            missing_count += 1
            if missing_count <= 5: # 只打印前5个错误，防止刷屏
                print(f"⚠️ 缺失: 找不到文件 {filename}")
            continue

        # --- B. 设定目标路径 (Destination) ---
        # 强制结构: Output / split / Category / filename
        dst_dir = os.path.join(OUTPUT_ROOT, split, folder_name)
        dst_path = os.path.join(dst_dir, filename)

        # --- C. 复制文件 ---
        try:
            os.makedirs(dst_dir, exist_ok=True)
            shutil.copy(src_path, dst_path)
            success_count += 1
        except Exception as e:
            print(f"❌ 复制失败 {filename}: {e}")

    # ================= 4. 结果统计 =================
    print("\n" + "="*30)
    print("处理完成！")
    print(f"✅ 成功复制: {success_count}")
    print(f"❌ 文件缺失: {missing_count}")
    print(f"📂 输出目录: {OUTPUT_ROOT}")
    
    # 打印目录结构确认
    print("\n生成的目录结构示例:")
    if success_count > 0:
        print(f"  {OUTPUT_ROOT}\\train\\Cargo\\0_1.wav")
        print(f"  {OUTPUT_ROOT}\\test\\Background\\4_0_1.wav")
        print("  (所有文件都直接位于类别文件夹下，无多余层级)")

In [10]:
# 运行主函数
split_dataset_unified()

📋 待处理任务总数: 28377
🔍 正在扫描源目录建立索引: X:\\数据集\DeepShip\\data_preprocessing\\data_audio_rename_add_enviromental_noise_class
   (如果文件很多，这可能需要几秒钟...)
✅ 索引建立完成！共找到 28377 个音频文件。


Splitting: 100%|██████████| 28377/28377 [03:30<00:00, 135.07it/s]


处理完成！
✅ 成功复制: 28377
❌ 文件缺失: 0
📂 输出目录: X:\\数据集\\DeepShip\\data_preprocessing\\data_audio_rename_add_enviromental_noise_class_dataset_split

生成的目录结构示例:
  X:\\数据集\\DeepShip\\data_preprocessing\\data_audio_rename_add_enviromental_noise_class_dataset_split\train\Cargo\0_1.wav
  X:\\数据集\\DeepShip\\data_preprocessing\\data_audio_rename_add_enviromental_noise_class_dataset_split\test\Background\4_0_1.wav
  (所有文件都直接位于类别文件夹下，无多余层级)
